In [154]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

## 1. Configuración inicial

In [155]:
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

In [156]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/df23.pickle', 'rb') as archivo:
    df = pickle.load(archivo)

Elimino columnas que no se utilizarán en el análisisÑ 
- ResponseId (index)
- Check (pregunta en la encuesta para verificar que el encuestado presta atencion a lo que responde)
- ConvertedCompYearly (conversion a dolares americanos de los que gana, lo elimino porque tratare solo EUR)

In [157]:
df = df.drop(columns=['Q120', 'ConvertedCompYearly', 'Country', 'Currency'])

### 2. Codificación de MainBranch
Empiezo a trabajar columna por columna con los ecoders. 
0. MainBranch: Que tipo de programador eres:
- I am a developer by profession
- I am not primarily a developer, but I write code sometimes as part of my work/studies 
- I used to be a developer by profession, but no longer am 
- I am learning to code 
- I code primarily as a hobby 
- None of these

Decido hacer un Label encoder, con valores del 0 (None of these) al 5 (Developer by profession), ya que quiero que se le de eventualmente mas peso a estos perfiles

In [158]:
df['MainBranch'].unique()

array(['I am a developer by profession',
       'I am not primarily a developer, but I write code sometimes as part of my work/studies'],
      dtype=object)

In [159]:
labels = {#'None of these': 0, 
        #   'I code primarily as a hobby': 1, 
        #   'I am learning to code':2,              --------------> todos estos son valores que aparecen en la encuesta, pero no en las respuestas
        #   'I used to be a developer by profession, but no longer am':3, 
          'I am not primarily a developer, but I write code sometimes as part of my work/studies': 4,
          'I am a developer by profession':5}
df['MainBranch'] = df['MainBranch'].map(labels).fillna(-1)

In [160]:
df['MainBranch'].isna().sum()

0

1. Age:
- Under 18 years old 
- 18-24 years old 
- 25-34 years old 
- 35-44 years old 
- 45-54 years old 
- 55-64 years old 
- 65 years or older 
- Prefer not to say

Decido hacer un OneHotEncoder: aunque se agregarian mas columnas, no son muchas. No quiero que se le de mas importancia a unas edades que a otras. 

Antes de hacer el OneHotEncoder verifico que no hayan nans. Si los hay, los trato primero, y leugo hago OneHot

In [161]:
print(df['Age'].isna().sum())
print(df['Age'].dtype)
print(df['Age'].unique())
df['Age'] = df['Age'].fillna('Prefer not to say')  # Rellenar NaN con una categoría válida
df['Age'] = df['Age'].str.strip()   # por las dudas, elimino espacios vacios que me puedan causar problemas con el One Hot

0
object
['35-44 years old' '18-24 years old' '25-34 years old' '45-54 years old'
 '55-64 years old' '65 years or older' 'Under 18 years old']


In [162]:
encoder = OneHotEncoder(sparse_output=False)

# Aplicamos el OneHotEncoder a la columna 'Age'
encoded = encoder.fit_transform(df[['Age']])

# Convertimos la matriz de salida a un DataFrame
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['Age']), index=df.index)

# Unimos el DataFrame original con las nuevas columnas codificadas
df = pd.concat([df, encoded_df], axis=1)

In [163]:
df = df.drop('Age', axis=1)

Compruebo que todo esta correcto, se han agregado las columnas correspondientes, y las filas siguen igual. Sigo. 

2. Employment:
- Employed, full-time 
- Employed, part-time 
- Independent contractor, freelancer, or self-employed
- Not employed, but looking for work 
- Not employed, and not looking for work 
- Student, full-time 
- Student, part-time 
- Retired 
- I prefer not to say

Decido usar un LabelEncoder y darle una mayor importancia al empleado full-time.

In [164]:
df.head()

,MainBranch,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,LearnCodeCoursesCert,YearsCode,YearsCodePro,DevType,OrgSize,PurchaseInfluence,TechList,BuyNewTool,CompTotal,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,OpSysPersonal use,OpSysProfessional use,OfficeStackAsyncHaveWorkedWith,OfficeStackAsyncWantToWorkWith,OfficeStackSyncHaveWorkedWith,OfficeStackSyncWantToWorkWith,AISearchHaveWorkedWith,AISearchWantToWorkWith,AIDevHaveWorkedWith,AIDevWantToWorkWith,NEWSOSites,SOVisitFreq,SOAccount,SOPartFreq,SOComm,SOAI,AISelect,AISent,AIAcc,AIBen,AIToolInterested in Using,AIToolCurrently Using,AIToolNot interested in Using,AINextVery different,AINextNeither different nor similar,AINextSomewhat similar,AINextVery similar,AINextSomewhat different,TBranch,ICorPM,WorkExp,Knowledge_1,Knowledge_2,Knowledge_3,Knowledge_4,Knowledge_5,Knowledge_6,Knowledge_7,Knowledge_8,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,ProfessionalTech,Industry,SurveyLength,SurveyEase,Age_18-24 years old,Age_25-34 years old,Age_35-44 years old,Age_45-54 years old,Age_55-64 years old,Age_65 years or older,Age_Under 18 years old
0,5,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Books / Physical media;School (i.e., Universit...",NaN,NaN,Less than 1 year,10,"Developer, back-end",2 to 9 employees,I have little or no influence,NaN,Start a free trial;Ask developers I know/work ...,35000.0,Clojure;PHP,Bash/Shell (all shells);HTML/CSS;JavaScript;Ko...,MariaDB;SQLite;Supabase,PostgreSQL,NaN,NaN,NaN,NaN,Capacitor,NaN,NaN,NaN,Emacs,Emacs;PhpStorm,Debian;Other Linux-based,Debian;Other Linux-based,NaN,NaN,Slack,Slack;Zoom,NaN,NaN,NaN,NaN,Stack Overflow,Multiple times per day,Yes,Less than once per month or monthly,"Yes, somewhat",NaN,Yes,Favorable,Increase productivity,Neither trust nor distrust,NaN,Learning about a codebase;Writing code;Documen...,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Individual contributor,10.0,Strongly agree,Agree,Agree,Agree,Agree,Agree,Agree,Agree,1-2 times a week,1-2 times a week,1-2 times a week,Less than 15 minutes a day,Less than 15 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",Too long,Neither easy nor difficult,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,5,"Employed, full-time",Remote,Hobby;Freelance/contract work,"Associate degree (A.A., A.S., etc.)","On the job training;School (i.e., University, ...",NaN,NaN,15,15,"Developer, back-end","10,000 or more employees",I have little or no influence,NaN,Ask developers I know/work with;Visit develope...,70000.0,Elixir;Go;Java;JavaScript;PHP;Ruby,Elixir;Go;Java;JavaScript;Kotlin;Rust,MySQL;PostgreSQL;Redis,NaN,Amazon Web Services (AWS);Digital Ocean;Google...,Amazon Web Services (AWS);Digital Ocean;Google...,jQuery;Node.js;Spring Boot;WordPress,Node.js;Phoenix;Spring Boot,NaN,NaN,Docker;Gradle;Homebrew;Kubernetes;Maven (build...,Docker;Gradle;Homebrew;Kubernetes;Maven (build...,IntelliJ IDEA;Vim;Visual Studio Code,IntelliJ IDEA;Vim;Visual Studio Code,MacOS;Windows,MacOS,Confluence;GitHub Discussions,GitHub Discussions,Skype;Slack;Zoom,Skype;Slack;Zoom,ChatGPT,ChatGPT,NaN,NaN,Stack Overflow,A few times per month or weekly,Yes,Less than once per month or monthly,"No, not really",Yes,"No, but I plan to soon",Indifferent,NaN,Neither trust nor distrust,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Individual contributor,15.0,Agree,Agree,Disagree,Agree,Agree,Agree,Neither agree nor disagree,Agree,1-2 times a week,1-2 times a week,1-2 times a week,15-30 minutes a day,30-60 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Developmen

In [165]:
df['Employment'].unique()

array(['Employed, full-time',
       'Employed, full-time;Independent contractor, freelancer, or self-employed',
       'Independent contractor, freelancer, or self-employed',
       'Employed, full-time;Independent contractor, freelancer, or self-employed;Employed, part-time',
       'Employed, part-time',
       'Independent contractor, freelancer, or self-employed;Employed, part-time',
       'I prefer not to say', 'Employed, full-time;Employed, part-time',
       'Retired'], dtype=object)

In [166]:
# Nuevo diccionario de mapeo
mapping = {
    'I prefer not to say': -1,
    'Retired': 0,
    'Not employed, and not looking for work': 0,
    'Not employed, but looking for work': 0,
    'Student, part-time': 0.5,
    'Student, full-time': 0,
    'Independent contractor, freelancer, or self-employed': 1,
    'Employed, part-time': 0.5,
    'Employed, full-time': 1
}

# Llenar los NaN con un valor temporal
df['Employment'].fillna('I prefer not to say', inplace=True)

# Función para asignar el valor mapeado
def map_employment(value):
    total_value = 0
    for key, val in mapping.items():
        if key in value:
            total_value += val
    return total_value

# Aplicar la función de mapeo
df['employment_map'] = df['Employment'].apply(map_employment)
df = df.drop(['Employment'], axis=1)

C:\Users\emita\AppData\Local\Temp\ipykernel_4612\846225123.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Employment'].fillna('I prefer not to say', inplace=True)


In [167]:
df['employment_map'].value_counts()

employment_map
 1.0    1073
 2.0      52
 1.5      12
 0.5      10
 2.5       1
-1.0       1
 0.0       1
Name: count, dtype: int64

3. RemoteWork:
- Remote 
- In-person 
- Hybrid (some remote, some in-person)

Decido un HoeHot (no son muchas columas, y quiero a todas con igualdad de importancia)

In [168]:
df['RemoteWork'].isna().sum()

2

In [169]:
encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(df[['RemoteWork']])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['RemoteWork']), index=df.index)
df = pd.concat([df, encoded_df], axis=1)

In [170]:
df = df.drop(['RemoteWork', 'RemoteWork_nan'], axis=1)

4. CodingActivities (fuera del trabajo):
- Hobby (2)
- Freelance/contract work (3)
- Contribute to open-source projects (4) 
- Bootstrapping a business (5)
- School or academic work (1)
- Professional development or self-paced learning from online courses (6)
- I don’t code outside of work (0)
- Other (-1)

Decido un Label encoder y darle mas peso a unas categorias, mas que a otras (indicado el peso de cada una)

Hay Muchos Nan, los trabajo imputandolos a la categoria de "Other"

In [171]:
df['CodingActivities'].isna().sum()

2

In [172]:
df['CodingActivities'] = df['CodingActivities'].fillna('Other')

In [173]:
labels4 = {
    'Hobby': 2,
    'Freelance/contract work': 3,
    'Contribute to open-source projects': 4,
    'Bootstrapping a business': 5,
    'School or academic work':1,
    'Professional development or self-paced learning from online courses': 6,
    'I don’t code outside of work': 0,
    'Other': -1
}
df['CodingActivities'] = df['CodingActivities'].map(labels4).fillna(-1)

In [174]:
df['CodingActivities'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1150 entries, 0 to 1149
Series name: CodingActivities
Non-Null Count  Dtype  
--------------  -----  
1150 non-null   float64
dtypes: float64(1)
memory usage: 9.1 KB


5. EdLevel: 
- Primary/elementary school (0)
- Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.) (1)
- Some college/university study without earning a degree (2)
- Associate degree (A.A., A.S., etc.) (3)
- Bachelor’s degree (B.A., B.S., B.Eng., etc.) (4) 
- Master’s degree (M.A., M.S., M.Eng., MBA, etc.) (5) 
- Professional degree (JD, MD, Ph.D, Ed.D, etc.) (6)
- Something else (-1)

Hago un LabelEncoder con los valores destacados

In [175]:
df['EdLevel'].unique()

array(['Master’s degree (M.A., M.S., M.Eng., MBA, etc.)',
       'Associate degree (A.A., A.S., etc.)',
       'Bachelor’s degree (B.A., B.S., B.Eng., etc.)',
       'Professional degree (JD, MD, Ph.D, Ed.D, etc.)',
       'Some college/university study without earning a degree',
       'Something else',
       'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
       'Primary/elementary school'], dtype=object)

In [176]:
df['EdLevel'].isna().sum()

0

In [177]:
labels5 = {
    'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)': 5,
    'Bachelor’s degree (B.A., B.S., B.Eng., etc.)': 4,
    'Professional degree (JD, MD, Ph.D, Ed.D, etc.)': 6,
    'Some college/university study without earning a degree': 2, 
    'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 1,
    'Associate degree (A.A., A.S., etc.)': 3, 
    'Something else': -1,
    'Primary/elementary school': 0
}
df['EdLevel'] = df['EdLevel'].map(labels5).fillna(-1)

6. LearnCode: 
- Books / Physical media (3): Implica un esfuerzo autodirigido, pero es formal y metódico.
- Coding Bootcamp (3): También estructurado, pero más intensivo y con un enfoque en habilidades prácticas.
- Colleague (2): Más relacionado con la práctica laboral, pero también limitado al entorno de trabajo.
- Friend or family member (1): Aprendizaje menos formal y probablemente limitado en alcance.
- Hackathons (virtual or in-person)
- Online Courses or Certification (2): Similar al entrenamiento en el trabajo, pero autodirigido y menos formal.
- On the job training (2): Aprender en el trabajo es práctico, pero menos estructurado.
- Other online resources (e.g., videos, blogs, forum, online community) (2): Aunque útil, suele ser más informal y variado en calidad.
- School (i.e., University, College, etc) (4): Representa una forma tradicional de aprendizaje, estructurada y generalmente reconocida como valiosa.
- Other (please specify):(0)

Le diversa importancia a las categorias con total discrecion (segun el tipo de educacion). Un OneHot me crearia demasiadas columnas. 

Prefiero hacer una funcion que, luego de dividir en el ";", se reemplace cada categoria por el numero correspondiente, y luego sumo esos numeros. De esta forma, una persona que se ha educado de 5 fuentes diversas, tendra mas puntaje que una que solo se ha educado con 1. Da cuenta del compromiso y profesionalismo que tiene esa persona. 

In [178]:
df['LearnCode'].isna().sum()   # hay un solo valor, lo dejamos a 0 (primera linea de la funcion)

6

Creo en un dict los valores

In [179]:
importance_dict = {
    'Books / Physical media': 3,
    'Coding Bootcamp': 3,
    'Colleague': 2,
    'Friend or family member': 1,
    'Hackathons (virtual or in-person)': 2,
    'Online Courses or Certification': 2,
    'On the job training': 2,
    'Other online resources (e.g., videos, blogs, forum, online community)': 2,
    'School (i.e., University, College, etc)': 4,
    'Other (please specify):': 0
}

Creo la funcion que lo divida en el ; y se le asigne a cada categoria un valor. Luego se suman todos los valores

In [180]:
def calculate_score(x):
    # Si el valor es NaN, devolvemos 0
    if pd.isna(x):
        return 0
    
    # Dividimos el texto en una lista de valores, usando ';' como separador
    values = x.split(';')
    
    # Inicializamos el puntaje en 0
    total_score = 0
    
    # Iteramos por cada valor en la lista
    for value in values:
        # Quitamos espacios en blanco alrededor del texto
        cleaned_value = value.strip()
        
        # Obtenemos el puntaje del diccionario y lo sumamos al puntaje total
        total_score += importance_dict.get(cleaned_value, 0)
    
    # Devolvemos el puntaje total
    return total_score

In [181]:
df['LearnCode'] = df['LearnCode'].apply(calculate_score)

7. LearnCodeOnline
    - Stack Overflow ('Community_Collaboration')
    - Blogs ('Community_Collaboration')
    - Online books ('Visual_Interactive')
    - Video-based online courses ('Visual_Interactive')
    - Online challenges (e.g., daily or weekly coding challenges) ('Community_Collaboration')
    - Coding sessions (live or recorded) ('Community_Collaboration')
    - Written-based online courses ('Technical_Reference')
    - How-to videos ('Visual_Interactive')
    - Auditory material (e.g., podcasts) ('Visual_Interactive')
    - Technical documentation ('Technical_Reference')
    - Certification videos ('Visual_Interactive')
    - Games that teach programming ('Visual_Interactive')
    - Written tutorials ('Technical_Reference')
    - Interactive tutorial ('Visual_Interactive')
    - Online forum ('Community_Collaboration')
    - Other ('Visual_Interactive')

Estas herramientas no tienen mas importancia unas que otras, ni son ordinales. Elijo usar OneHotEncoder. Como son muchas columnas, decido agruparlas, para que no sean tantas.
Dejo 3 categorias: 'Visual_Interactive', 'Technical_Reference', 'Community_Collaboration'. 

Creo una funcion para ver los valores unicos, ya que en las celdas hay muchos valores. 

In [182]:
def get_unique_values(df, column_name):
    # Inicializamos un conjunto vacío para almacenar valores únicos
    unique_values = set()

    # Iteramos sobre cada fila en la columna
    for value in df[column_name]:
        # Si el valor no es nulo (NaN)
        if pd.notna(value):
            # Si el valor contiene múltiples elementos separados por ';'
            if isinstance(value, str) and ';' in value:
                # Dividimos el string y añadimos cada elemento al conjunto
                unique_values.update(value.split(';'))
            else:
                # Si es un valor único, lo añadimos directamente al conjunto
                unique_values.add(value)

    # Convertimos el conjunto a una lista y la ordenamos
    return sorted(list(unique_values))

In [183]:
unique_list = get_unique_values(df, 'LearnCodeOnline')

unique_list

['Auditory material (e.g., podcasts)',
 'Blogs with tips and tricks',
 'Books',
 'Certification videos',
 'Click to write Choice 20',
 'Formal documentation provided by the owner of the tech',
 'How-to videos',
 'Interactive tutorial',
 'Online challenges (e.g., daily or weekly coding challenges)',
 'Other (Please specify):',
 'Programming Games',
 'Recorded coding sessions',
 'Stack Overflow',
 'Video-based Online Courses',
 'Written Tutorials',
 'Written-based Online Courses']

Mapeo las categorias para que no sean tantas al hacer OneHot

In [184]:
# Primero, creamos un diccionario para mapear las herramientas a las categorías
category_map = {
    'Stack Overflow': 'Technical_Reference',
    'Blogs': 'Technical_Reference',
    'Social Media': 'Technical_Reference',
    'Video-based online courses': 'Visual_Interactive',
    'Online challenges (e.g., daily or weekly coding challenges)': 'Technical_Reference',
    'Coding sessions (live or recorded)': 'Technical_Reference',
    'Written-based online courses': 'Technical_Reference',
    'How-to videos': 'Visual_Interactive',
    'Auditory material (e.g., podcasts)': 'Visual_Interactive',
    'Technical documentation': 'Technical_Reference',
    'Certification videos': 'Visual_Interactive',
    'Books': 'Technical_Reference',
    'Games': 'Visual_Interactive',
    'AI': 'Technical_Reference',
    'Written tutorials': 'Technical_Reference',
    'Interactive tutorial': 'Visual_Interactive',
    'Other (Please specify):': 'Visual_Interactive'
}
# Lleno los Nan con 'Other'
df['LearnCodeOnline'] = df['LearnCodeOnline'].fillna('Other (Please specify):')
# Aplicamos el mapeo a la columna 'LearnCodeOnline'
df['LearnCodeOnline_Category'] = df['LearnCodeOnline'].map(category_map)

In [185]:
# Creamos el OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Aplicamos el OneHotEncoder
encoded = encoder.fit_transform(df[['LearnCodeOnline_Category']])

# Creamos un DataFrame con las columnas codificadas
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['LearnCodeOnline_Category']), index=df.index)

# Unimos el DataFrame original con las nuevas columnas codificadas
df = pd.concat([df, encoded_df], axis=1)

# Eliminamos la columna temporal 'LearnCodeOnline_Category' si ya no la necesitas
df = df.drop(['LearnCodeOnline_Category', 'LearnCodeOnline', 'LearnCodeOnline_Category_nan'], axis=1)

8. 'LearnCodeCoursesCert': la elimino, no tiene mucha información para aportar.

In [186]:
df = df.drop(['LearnCodeCoursesCert'], axis=1)

9. YearsCode: sacar los nan y pasarlo a int

In [187]:
df['YearsCode'].isnull().sum()
moda = df['YearsCode'].mode()[0]
df['YearsCode'] = df['YearsCode'].fillna(moda)

In [188]:
df['YearsCode'] = df['YearsCode'].replace('Less than 1 year', 0)

In [189]:
df['YearsCode'] = df['YearsCode'].replace('More than 50 years', 50)

In [190]:
df['YearsCode']=df['YearsCode'].astype(int)

In [191]:
df['YearsCode'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1150 entries, 0 to 1149
Series name: YearsCode
Non-Null Count  Dtype
--------------  -----
1150 non-null   int32
dtypes: int32(1)
memory usage: 4.6 KB


10. YearsCodePro: reemplazo nan y paso a int

In [192]:
df['YearsCodePro'].isnull().sum()

8

In [193]:
moda = df['YearsCodePro'].mode()[0]
df['YearsCodePro'] = df['YearsCodePro'].fillna(moda)

In [194]:
df['YearsCodePro'].unique()

array(['10', '15', '3', '2', '13', '8', '11', '23', '30', '12', '6', '5',
       '7', '18', '20', '1', '9', '22', '28', '4', 'Less than 1 year',
       '25', '19', '17', '21', '14', '26', '27', '16', '24', '29', '35',
       '38', '34', '45', '37', '40', '31', '36', '41', '39', '33', '32',
       '49'], dtype=object)

In [195]:
df['YearsCodePro']=df['YearsCodePro'].replace('Less than 1 year', 0)

In [196]:
df['YearsCodePro']=df['YearsCodePro'].astype(int)
df['YearsCodePro'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1150 entries, 0 to 1149
Series name: YearsCodePro
Non-Null Count  Dtype
--------------  -----
1150 non-null   int32
dtypes: int32(1)
memory usage: 4.6 KB


11. DevType
- Academic researcher (5)
- Blockchain (6)
- Cloud infrastructure engineer (3)
- Data or business analyst (2)
- Data engineer (2)
- Data scientist or machine learning specialist (2)
- Database administrator (7)
- Designer (7)
- Developer Advocate (1)
- Developer, AI (1)
- Developer, back-end (1)
- Developer, desktop or enterprise applications (1)
- Developer, embedded applications or devices (1)
- Developer Experience (1)
- Developer, front-end (1)
- Developer, full-stack(1)
- Developer, game or graphics (1)
- Developer, mobile(1) 
- Developer, QA or test (1)
- DevOps specialist (3)
- Educator (5)
- Engineer, site reliability (3)
- Engineering manager (4)
- Hardware Engineer (6)
- Marketing or sales professional (7)
- Product manager (4)
- Project manager (4)
- Research & Development role (5)
- Scientist (5)
- Senior Executive (C-Suite, VP, etc.) (4)
- Student (7)
- System administrator (3)
- Security professional (3)
- Other (7)

Elijo agruparlos para que no sean tantos, y asi simplificar el analisis, y luego un OneHotEncoder:
1. Gestión y Liderazgo
2. Desarrollo de Software
3. Infraestructura y Sistemas
4. Ingeniería de Datos y Ciencia de Datos
5. Investigación y Educación
6. Hardware y Blockchain
7. Roles de Soporte y Otros

In [197]:
# Primero me fijo que no hayan Nan
df['DevType'].isnull().sum()

4

In [198]:
df['DevType']=df['DevType'].fillna('Other (please specify):')

In [199]:
df['DevType'].unique()

array(['Developer, back-end', 'Developer, full-stack',
       'Developer, front-end', 'Product manager',
       'Engineer, site reliability', 'Research & Development role',
       'Data scientist or machine learning specialist',
       'Cloud infrastructure engineer', 'Developer, mobile',
       'Data or business analyst',
       'Developer, desktop or enterprise applications',
       'DevOps specialist', 'Database administrator', 'Project manager',
       'Developer, embedded applications or devices',
       'Developer, game or graphics', 'Engineer, data',
       'Other (please specify):', 'Senior Executive (C-Suite, VP, etc.)',
       'Engineering manager', 'Academic researcher',
       'Developer Experience', 'Hardware Engineer', 'Blockchain',
       'Developer Advocate', 'System administrator', 'Educator',
       'Designer', 'Developer, QA or test', 'Scientist',
       'Security professional'], dtype=object)

In [200]:
# Primero, creamos un diccionario para mapear las herramientas a las categorías
dev_type_map = {
    'Academic researcher': 'Investigación y Educación',
    'Blockchain': 'Hardware y Blockchain',
    'Cloud infrastructure engineer': 'Infraestructura y Sistemas',
    'Data engineer': 'Ingeniería de Datos y Ciencia de Datos',
    'Data or business analyst': 'Ingeniería de Datos y Ciencia de Datos',
    'Data scientist or machine learning specialist': 'Ingeniería de Datos y Ciencia de Datos',
    'Designer': 'Roles de Soporte y Otros',
    'DevOps specialist': 'Infraestructura y Sistemas',
    # 'Developer Advocate': 'Desarrollo de Software',
    # 'Developer Experience': 'Desarrollo de Software',
    # 'Developer, AI': 'Desarrollo de Software',
    'Developer, QA or test': 'Desarrollo de Software',
    'Developer, back-end': 'Desarrollo de Software',
    'Developer, desktop or enterprise applications': 'Desarrollo de Software',
    'Developer, embedded applications or devices': 'Desarrollo de Software',
    'Developer, front-end': 'Desarrollo de Software',
    'Developer, full-stack': 'Desarrollo de Software',
    'Developer, game or graphics': 'Desarrollo de Software',
    'Developer, mobile': 'Desarrollo de Software',
    'Educator': 'Investigación y Educación',
    'Engineer, site reliability': 'Infraestructura y Sistemas',
    'Engineering manager': 'Gestión y Liderazgo',
    'Hardware Engineer': 'Hardware y Blockchain',
    'Other (please specify):': 'Roles de Soporte y Otros',
    'Product manager': 'Gestión y Liderazgo',
    'Project manager': 'Gestión y Liderazgo',
    'Research & Development role': 'Investigación y Educación',
    'Scientist': 'Investigación y Educación',
    'Security professional': 'Infraestructura y Sistemas',
    'Senior Executive (C-Suite, VP, etc.)': 'Gestión y Liderazgo',
    'System administrator': 'Infraestructura y Sistemas',
    # 'Marketing or sales professional': 'Roles de Soporte y Otros',
    # 'Student': 'Roles de Soporte y Otros',
    'Database administrator': 'Roles de Soporte y Otros'    
}
len(dev_type_map)

29

In [201]:
# Aplicamos el mapeo a la columna 'DevType'
df['DevType_Category'] = df['DevType'].map(dev_type_map)

# Creamos el OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Aplicamos el OneHotEncoder
encoded = encoder.fit_transform(df[['DevType_Category']])

# Creamos un DataFrame con las columnas codificadas
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['DevType_Category']), index=df.index)

# Unimos el DataFrame original con las nuevas columnas codificadas
df = pd.concat([df, encoded_df], axis=1)

# Eliminamos la columna temporal 'DevType_Category' si ya no la necesitas
df = df.drop(['DevType_Category', 'DevType', 'DevType_Category_nan'], axis=1)

12. OrgSize: Cuántas personas forman parte de la organización para la que trabajas (organización primaria)
- I don’t know
- Just me - I am a freelancer, sole proprietor, etc. 
- 2 to 9 employees 
- 10 to 19 employees 
- 20 to 99 employees 
- 100 to 499 employees 
- 500 to 999 employees 
- 1000 to 4999 employees 
- 5000 to 9999 employees 
- 10000 or more employees 

Label Encoder, en ese orden

In [202]:
df['OrgSize'].unique()

array(['2 to 9 employees', '10,000 or more employees',
       '500 to 999 employees', '1,000 to 4,999 employees',
       '20 to 99 employees', '100 to 499 employees', '10 to 19 employees',
       'Just me - I am a freelancer, sole proprietor, etc.',
       'I don’t know', '5,000 to 9,999 employees', nan], dtype=object)

In [203]:
# Primero trato los Nan
df['OrgSize'] = df['OrgSize'].fillna('I don’t know')
print(df['OrgSize'].isna().sum())

0


In [204]:
type(df.iloc[4, 8])

float

Los valores NaN siguen apareciendo, a pesar de hacer utilizado el fillna mas arriba. Por eso lo pongo tambien al hacer el map. 

In [205]:
labels12 = {
    'I don’t know': -1,
    'Just me - I am a freelancer, sole proprietor, etc.': 0,
    '2 to 9 employees': 1,
    '10 to 19 employees': 2,
    '20 to 99 employees': 3,
    '100 to 499 employees': 4,
    '500 to 999 employees': 5, 
    '1,000 to 4,999 employees': 6, 
    '5,000 to 9,999 employees': 7,
    '10,000 or more employees': 8 
}
df['OrgSize'] = df['OrgSize'].map(labels12).fillna(-1)

13. PurchaseInfluence: Nivel de influencia sobre nuevas tecnologías adquiridas por la organización
- I have little or no influence 
- I have some influence 
- I have a great deal of influence

Label Encoder en ese orden

In [206]:
# Primero trato los Nan
df['PurchaseInfluence'] = df['PurchaseInfluence'].fillna('Other')
print(df['PurchaseInfluence'].isna().sum())

0


In [207]:
df['PurchaseInfluence'].unique()

array(['I have little or no influence', 'I have some influence',
       'I have a great deal of influence', 'Other'], dtype=object)

In [208]:
labels13 = {
    'I have little or no influence': 0,
    'I have some influence': 1,
    'I have a great deal of influence': 2,
    'Other': -1
}
df['PurchaseInfluence'] = df['PurchaseInfluence'].map(labels13).fillna(-1)

14. TechList: Irrelevante, lo elimino

In [209]:
df = df.drop(['TechList'], axis=1)

15. BuyNewTool: Cómo descubrir o investigas soluciones disponibles en nuevas herramientas o softwares
- Start a free trial 
- Ask developers I know/work with 
- Ask a generative AI tool 
- Visit developer communities like Stack Overflow 
- Read ratings or reviews on third party sites like G2 Crowd 
- Research companies that have advertised on sites I visit 
- Research companies that have emailed me
- Other 

No tienen orden, separo por ; y hago OneHotEncoder al final

In [210]:
df['BuyNewTool'].unique()

array(['Start a free trial;Ask developers I know/work with',
       'Ask developers I know/work with;Visit developer communities like Stack Overflow',
       'Start a free trial;Ask developers I know/work with;Visit developer communities like Stack Overflow',
       nan,
       'Start a free trial;Ask developers I know/work with;Visit developer communities like Stack Overflow;Ask a generative AI tool',
       'Start a free trial;Ask developers I know/work with;Visit developer communities like Stack Overflow;Read ratings or reviews on third party sites like G2 Crowd',
       'Ask developers I know/work with',
       'Ask developers I know/work with;Visit developer communities like Stack Overflow;Read ratings or reviews on third party sites like G2 Crowd',
       'Start a free trial;Ask developers I know/work with;Visit developer communities like Stack Overflow;Read ratings or reviews on third party sites like G2 Crowd;Ask a generative AI tool',
       'Visit developer communities like S

In [211]:
df['BuyNewTool'].isna().sum()
df['BuyNewTool'] = df['BuyNewTool'].fillna('Other')

#### Ahora vienen una seguidilla de columnas que tengo que separar por el ';' y luego hacer un OneHotEncoder. 

Creo una funcion para aplicar lo mismo a todas juntas. El codigo de estas columnas se vera al final.

Luego elimino las que indico en el listado, por no aportar informacion nueva ni relevante

16. LanguageHaveWorkedWith: Dividirlo por ';' y luego OneHotEncoder. Se haran muchas columnas, pero no queda otra. 

17. LanguageWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

18. DatabaseHaveWorkedWith: Igual

19. DatabaseWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

20. PlatformHaveWorkedWith: Igual

21. PlatformWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

22. WebframeHaveWorkedWith: Igual

23. WebframeWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

24. MiscTechHaveWorkedWith: Igual 

25. MiscTechWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas 

26. ToolsTechHaveWorkedWith: Igual 

27. ToolsTechWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

28. NEWCollabToolsHaveWorkedWith: Igual 

29. NEWCollabToolsWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

30. OpSysPersonal use: La elimino, me quedo solo con el uso de sistema operativo profesional. 

31. OpSysProfessional use: Agrupo los sistemas operativos en los mas representativos para no hacer tantas columnas: 
- AIX (4) Otros
- Android (3) Android-based
- Arch (1)
- BSD (1)
- ChromeOS (3)
- Cygwin (4)
- Debian (1) Linux-based
- Fedora (1)
- Haiku (1)
- iOS (2) MacOs
- iPadOS (2)
- MacOS (2)
- Other Linux-based (1)
- Red Hat (1)
- Solaris (1)
- Ubuntu (1)
- Windows (0) Windows-based
- Windows Subsystem for Linux (WSL) (0)
- Other (4)

Una vez agrupadas, le hago un MultilabelEncoder para poder poner varias etiquetas por fila

In [212]:
df['OpSysProfessional use'].unique()

array(['Debian;Other Linux-based', 'MacOS',
       'Windows;Windows Subsystem for Linux (WSL)', nan, 'Fedora',
       'Ubuntu', 'Windows', 'Fedora;Windows', 'MacOS;Windows',
       'MacOS;Ubuntu', 'Ubuntu;Windows', 'Arch;Debian;Ubuntu',
       'iOS;iPadOS;MacOS', 'Android;iOS;MacOS', 'Android;MacOS', 'iOS',
       'Debian;Other Linux-based;Ubuntu', 'iOS;MacOS',
       'Android;iOS;iPadOS;MacOS;Ubuntu',
       'Arch;Debian;Ubuntu;Windows;Windows Subsystem for Linux (WSL)',
       'Arch;Windows;Windows Subsystem for Linux (WSL)', 'iPadOS;MacOS',
       'Ubuntu;Windows;Windows Subsystem for Linux (WSL)',
       'Android;Debian;Other Linux-based;Ubuntu;Windows', 'Arch',
       'Debian;Other Linux-based;Ubuntu;Windows;Windows Subsystem for Linux (WSL)',
       'Other (Please Specify):', 'Android;Windows',
       'Debian;Ubuntu;Windows;Windows Subsystem for Linux (WSL)',
       'Other Linux-based;Red Hat',
       'Cygwin;Windows;Windows Subsystem for Linux (WSL)',
       'MacOS;Ubuntu;Window

In [213]:
# Primero trato los Nan
df['OpSysProfessional use'] = df['OpSysProfessional use'].fillna('Other')
df['OpSysProfessional use'].isnull().sum()

0

In [214]:
# Diccionario de mapeo
os_map = {
    'AIX': 'Otros',
    'Android': 'Android-based',
    'Arch': 'Linux-based',
    'BSD': 'Linux-based',
    'ChromeOS': 'Android-based',
    'Cygwin': 'Otros',
    'Debian': 'Linux-based',
    'Fedora': 'Linux-based',
    'Haiku': 'Linux-based',
    'iOS': 'MacOS',
    'iPadOS': 'MacOS',
    'MacOS': 'MacOS',
    'Other Linux-based': 'Linux-based',
    'Red Hat': 'Linux-based',
    'Solaris': 'Linux-based',
    'Ubuntu': 'Linux-based',
    'Windows': 'Windows-based',
    'Windows Subsystem for Linux (WSL)': 'Windows-based',
    'Other': 'Otros'
}

In [215]:
# Función modificada para manejar NaN
def split_and_map(value):
    if pd.isna(value):
        return []  # Retorna una lista vacía para NaN
    return [os_map.get(os.strip(), 'Otros') for os in value.split(';')]

# Aplicar la función a la columna
df['OpSysProfessional_Category'] = df['OpSysProfessional use'].apply(split_and_map)

# Usar MultiLabelBinarizer para el one-hot encoding
mlb = MultiLabelBinarizer()
encoded = mlb.fit_transform(df['OpSysProfessional_Category'])

# Crear DataFrame con las columnas codificadas
encoded_df = pd.DataFrame(encoded, columns=mlb.classes_, index=df.index)

# Unir el DataFrame original con las nuevas columnas codificadas
df = pd.concat([df, encoded_df], axis=1)

# Eliminar las columnas originales si ya no las necesitas
df = df.drop(['OpSysProfessional_Category', 'OpSysProfessional use'], axis=1)

# Verificar que no se haya creado una columna 'nan'
if 'nan' in encoded_df.columns:
    df = df.drop('nan', axis=1)

32. OfficeStackAsyncHaveWorkedWith (Herramientas de documentación que has usado): Igual 

33. OfficeStackAsyncWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

34. OfficeStackSyncHaveWorkedWith(Herramientas de comunicación que has usado): Igual

35. OfficeStackSyncWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

36. AISearchDevHaveWorkedWith: Igual

37. AISearchWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitiva

38. AIDevHaveWorkWith: Igual

39. AIDevWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

40. NEWSOSites(Uso de StackOverFlow): No tiene relevancia para mi analisis en concreto, lo elimino. 

41. SOVisitFreq: No tiene relevancia para mi analisis en concreto, lo elimino.

42. SOAccount: No tiene relevancia para mi analisis en concreto, lo elimino.

43. SOPartFreq: No tiene relevancia para mi analisis en concreto, lo elimino.

44. SOComm: No tiene relevancia para mi analisis en concreto, lo elimino.

45. SOAI: No tiene relevancia para mi analisis en concreto, lo elimino.

46. AISelect (Usas IA para tu proceso de desarrollo):
- Yes 
- No, but I plan to soon 
- No, and I don't plan to
Elijo Label encoder con (2) para el Yes.

In [216]:
df['AISelect'].unique()

array(['Yes', 'No, but I plan to soon', "No, and I don't plan to"],
      dtype=object)

In [217]:
# Primero trato los Nan
df['AISelect'] = df['AISelect'].fillna('Other')
print(df['AISelect'].isna().sum())

0


In [218]:
labels60 = {
    "No, and I don't plan to": 0,
    'No, but I plan to soon': 1,
    'Yes': 2,
    'Other': -1
}
df['AISelect'] = df['AISelect'].map(labels60).fillna(-1)

47. AISent (Uso de herramientas de IA como parte de su flujo de trabajo de desarrollo): 
- Very favorable (5) 
- Favorable(4)
- Indifferent (3)
- Unfavorable (2)
- Very unfavorable (1)
- Unsure (0)

LabelEncoder con los valores indicados. 

In [219]:
df['AISent'].unique()

array(['Favorable', 'Indifferent', 'Unsure', nan, 'Very favorable',
       'Unfavorable', 'Very unfavorable'], dtype=object)

In [220]:
# Primero trato los Nan
df['AISent'] = df['AISent'].fillna('Unsure')
print(df['AISent'].isna().sum())

0


In [221]:
labels61 = {
    'Very favorable': 5, 
    'Favorable': 4, 
    'Indifferent': 3, 
    'Unfavorable': 2,
    'Very unfavorable': 1,
    'Unsure': 0
}
df['AISent'] = df['AISent'].map(labels61).fillna(-1)

48. AIAcc (Cuanto confias en el output de la IA como parte de tu trabajo):
- Increase productivity
- Greater efficiency
- Improve collaboration
- Speed up learning
- Improve accuracy in coding
- None of the above

Dividirlo por ';' y luego OneHotEncoder. 

49. AIBen (beneficios importantes espera lograr con la IA): 
- Increase productivity 
- Greater efficiency 
- Make workload more manageable 
- Improve collaboration 
- Speed up learning 
- Improve accuracy in coding 
- None of the above 
- Other 

Dividirlo por ';' y luego OneHotEncoder. 

50. AIToolInterested in Using: La elimino, dejo solo la de "Currently in using" como representativa. Sino se me hacen muchas columnas y muy repetitivas

51. AIToolCurrently Using:
- Learning about a codebase 
- Project planning 
- Writing code 
- Documenting code 
- Debugging and getting help 
- Testing code 
- Committing and reviewing code 
- Deployment and monitoring 
- Predictive analytics 
- Search for answers 
- Generating content or synthetic data 
- Other 

Dividirlo por ';' y luego OneHotEncoder.

52. AIToolNot interested in Using: La elimino, dejo solo la de "Currently in using" como representativa. Sino se me hacen muchas columnas y muy repetitivas

53. AINextVery different: La elimino, dejo solo la de "Currently in using" como representativa. Sino se me hacen muchas columnas y muy repetitivas

54. AINextNeither different nor similar: La elimino, dejo solo la de "Currently in using" como representativa. Sino se me hacen muchas columnas y muy repetitivas

55. AINextSomewhat similar: La elimino, dejo solo la de "Currently in using" como representativa. Sino se me hacen muchas columnas y muy repetitivas

56. AINextVery similar: La elimino, dejo solo la de "Currently in using" como representativa. Sino se me hacen muchas columnas y muy repetitivas

57. AINextSomewhat different: La elimino, dejo solo la de "Currently in using" como representativa. Sino se me hacen muchas columnas y muy repetitivas

58. TBranch: Irrelevante, la elimino

59. ICorPM: Irrelevante, la elimino

60. WorkExp: Los dejo como estan 

61. Knowledge_1: Irrelevante, lo elimino

62. Knowledge_2: Irrelevante, lo elimino

63. Knowledge_3: Irrelevante, lo elimino

64. Knowledge_4: Irrelevante, lo elimino

65. Knowledge_5: Irrelevante, lo elimino

66. Knowledge_6: Irrelevante, lo elimino

67. Knowledge_7: Irrelevante, lo elimino

68. Knowledge_8: Irrelevante, lo elimino

69. Frequency_1: (Qué tan frecuente experimentas ‘Needing help from people outside of your immediate team?‘): LabelEncoder 

In [222]:
# Primero trato los Nan
df['Frequency_1'] = df['Frequency_1'].fillna('Other')
print(df['Frequency_1'].isna().sum())

0


In [223]:
df['Frequency_1'].unique()

array(['1-2 times a week', 'Never', 'Other', '10+ times a week',
       '3-5 times a week', '6-10 times a week'], dtype=object)

In [224]:
labels88 = {
    '10+ times a week': 4, 
    '6-10 times a week': 3, 
    '3-5 times a week': 2,
    '1-2 times a week': 1, 
    'Never': 0, 
    'Other': -1
}
df['Frequency_1'] = df['Frequency_1'].map(labels88).fillna(-1)

70. Frequency_2 (Qué tan frecuente experimentas ‘Interacting with people outside of your immediate team?’):  LabelEncoder

Mismas categorias que la 88, uso ese labels

In [225]:
df['Frequency_2'].unique()

array(['1-2 times a week', nan, 'Never', '6-10 times a week',
       '3-5 times a week', '10+ times a week'], dtype=object)

In [226]:
# Primero trato los Nan
df['Frequency_2'] = df['Frequency_2'].fillna('Other')
print(df['Frequency_2'].isna().sum())

0


In [227]:
df['Frequency_2'] = df['Frequency_2'].map(labels88).fillna(-1)

71. Frequency_3	(Qué tan frecuente experimentas ‘Encountering knowledge silos (where one individual or team has information that's not shared or distributed with other individuals or teams) at work?’): LabelEncoder

Mismas categorias que la 88, uso ese labels

In [228]:
df['Frequency_3'].unique()

array(['1-2 times a week', 'Never', nan, '6-10 times a week',
       '3-5 times a week', '10+ times a week'], dtype=object)

In [229]:
# Primero trato los Nan
df['Frequency_3'] = df['Frequency_3'].fillna('Other')
print(df['Frequency_3'].isna().sum())

0


In [230]:
df['Frequency_3'] = df['Frequency_3'].map(labels88).fillna(-1)

72. TimeSearching: (Promedio de tiempo que gastas buscando respuesta o solución a algún problema): 
- 'Less than 15 minutes a day': 0,
- '15-30 minutes a day': 1,
- '30-60 minutes a day': 2,
- '60-120 minutes a day': 3,
- 'Over 120 minutes a day': 4

LabelEncoder con los valores asignados

In [231]:
df['TimeSearching'].unique()

array(['Less than 15 minutes a day', '15-30 minutes a day',
       '30-60 minutes a day', nan, '60-120 minutes a day',
       'Over 120 minutes a day'], dtype=object)

In [232]:
# Primero trato los Nan
df['TimeSearching'] = df['TimeSearching'].fillna('IDK')
print(df['TimeSearching'].isna().sum())

0


In [233]:
labels91 = {
'Less than 15 minutes a day': 0,
    '15-30 minutes a day': 1,
    '30-60 minutes a day': 2,
    '60-120 minutes a day': 3,
    'Over 120 minutes a day': 4,
    'IDK': -1
}
df['TimeSearching'] = df['TimeSearching'].map(labels91).fillna(-1)

73. TimeAnswering	



In [234]:
df['TimeAnswering'].unique()

array(['Less than 15 minutes a day', '30-60 minutes a day',
       '15-30 minutes a day', nan, '60-120 minutes a day',
       'Over 120 minutes a day'], dtype=object)

In [235]:
# Primero trato los Nan
df['TimeAnswering'] = df['TimeAnswering'].fillna('IDK')
print(df['TimeAnswering'].isna().sum())

0


In [236]:
df['TimeAnswering'] = df['TimeAnswering'].map(labels91).fillna(-1)

74. ProfessionalTech:  Irrelevante, lo elimino.

75. Industry (Industria de tu compañía):
- Software Development (0)
- Computer Systems Design and Services (0)
- Internet, Telecomm or Information Services (0)
- Fintech (0)
- Energy (3)
- Government (4)
- Banking/Financial Services (1)
- Manufacturing (3)
- Transportation, or Supply Chain (3)
- Healthcare (2)
- Retail and Consumer Services (4)
- Higher Education (2)
- Media & Advertising Services (4)
- Insurance (1)
- Other(4)

Los agrupo y les hago OneHotEncoder. 

In [237]:
df['Industry'].unique()

array(['Information Services, IT, Software Development, or other Technology',
       nan, 'Other', 'Healthcare', 'Retail and Consumer Services',
       'Legal Services', 'Higher Education', 'Financial Services',
       'Advertising Services',
       'Manufacturing, Transportation, or Supply Chain', 'Insurance',
       'Wholesale', 'Oil & Gas'], dtype=object)

In [238]:
# Diccionario de mapeo
industry_map = {
    'Information Services, IT, Software Development, or other Technology': 'Tecnología y Servicios Digitales',
    'Other:': 'Otros Servicios', 
    'Healthcare': 'Salud y Educación',
    'Retail and Consumer Services': 'Otros Servicios',
    'Legal Services': 'Otros Servicios',
    'Higher Education': 'Salud y Educación',
    'Financial Services': 'Servicios Financieros',
    'Advertising Services': 'Otros Servicios',
    'Manufacturing, Transportation, or Supply Chain': 'Industria y Energía',
    'Insurance': 'Servicios Financieros',
    'Wholesale': 'Otros Servicios',
    'Oil & Gas': 'Industria y Energía'
}

# Función para manejar NaN y aplicar el mapeo
def map_industry(value):
    if pd.isna(value):
        return 'Desconocido'
    return industry_map.get(value, 'Otros Servicios')

# Suponiendo que tienes un DataFrame llamado 'df' con una columna 'Industry'
# Aplicamos el mapeo a la columna 'Industry'
df['Industry_Category'] = df['Industry'].map(industry_map)

# Creamos el OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Aplicamos el OneHotEncoder
encoded = encoder.fit_transform(df[['Industry_Category']])

# Creamos un DataFrame con las columnas codificadas
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['Industry_Category']), index=df.index)

# Unimos el DataFrame original con las nuevas columnas codificadas
df = pd.concat([df, encoded_df], axis=1)

# Eliminamos la columna temporal 'Industry_Category' si ya no la necesitas
df = df.drop(['Industry_Category', 'Industry', 'Industry_Category_nan'], axis=1)

Por la forma en que OneHotEncoder trata los Nan, se me siguen creando columnas con Industry_Category_nan. Simplemente la Elimino y esa fila quedara sin valores

Creo la funcion para aplicar OneHotEncoder a todas las columnas que quedan, a las que primero tengo que tratar haciendo split de las ;. 

In [239]:
def process_and_encode(df, columns):
    for column in columns:
        # Separamos las categorías por el delimitador ";"
        df[column] = df[column].fillna('').str.split(';')
        
        # Creamos un objeto MultiLabelBinarizer
        mlb = MultiLabelBinarizer()
        
        # Aplicamos MultiLabelBinarizer a las categorías separadas
        encoded_values = mlb.fit_transform(df[column])
        
        # Creamos un DataFrame con los valores codificados
        encoded_df = pd.DataFrame(encoded_values, columns=[f"{column}_{c}" for c in mlb.classes_], index=df.index)
        
        # Añadimos los resultados al DataFrame original
        df = pd.concat([df, encoded_df], axis=1)
        
        # Eliminamos la columna original
        df.drop(column, axis=1, inplace=True)
    
    return df

Los Nan que existan en estas columnas he decidido dejarlos, y cuando se aplique el OneHotEncoder se pondran todos 0 en las columnas que agrega. En caso de que no sea asi, eliminare las columnas que sean nan

In [240]:
# Supongamos que estas son las columnas que contienen valores separados por ';'
columns_to_encode = ['BuyNewTool',
    'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith', 
    'MiscTechHaveWorkedWith', 'ToolsTechHaveWorkedWith', 'NEWCollabToolsHaveWorkedWith', 'OfficeStackAsyncHaveWorkedWith', 'OfficeStackSyncHaveWorkedWith',
    'AISearchHaveWorkedWith', 'AIDevHaveWorkedWith', 'AIAcc',
    'AIBen', 'AIToolCurrently Using'
    ]  # Reemplaza con los nombres reales de las columnas


# Aplicamos la función al DataFrame
df_encoded = process_and_encode(df, columns_to_encode)

76. SurveyLength:

78. SurveyEase: Irrelevante, lo elimino.

In [241]:
columns_to_drop = [
    'LanguageWantToWorkWith', 'DatabaseWantToWorkWith', 'PlatformWantToWorkWith', 'WebframeWantToWorkWith', 'MiscTechWantToWorkWith', 'ToolsTechWantToWorkWith', 'NEWCollabToolsWantToWorkWith', 'OpSysPersonal use',
    'OfficeStackAsyncWantToWorkWith', 'OfficeStackSyncWantToWorkWith', 'AISearchWantToWorkWith', 'AIDevWantToWorkWith',
    'NEWSOSites', 'SOVisitFreq', 'SOAccount', 'SOPartFreq', 'SOComm', 'SOAI',
    'AIToolInterested in Using', 'AIToolNot interested in Using', 'AINextVery different', 
    'AINextNeither different nor similar', 'AINextSomewhat similar', 'AINextVery similar', 'AINextSomewhat different',
    'TBranch', 'ICorPM', 'WorkExp', 
    'Knowledge_1', 'Knowledge_2', 'Knowledge_3', 'Knowledge_4', 'Knowledge_5', 'Knowledge_6', 'Knowledge_7', 'Knowledge_8', 
    'ProfessionalTech', 'SurveyLength', 'SurveyEase'
]

In [242]:
df = df.drop(columns_to_drop, axis=1)

In [243]:
df=df.drop(['BuyNewTool',
    'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith', 
    'MiscTechHaveWorkedWith', 'ToolsTechHaveWorkedWith', 'NEWCollabToolsHaveWorkedWith', 'OfficeStackAsyncHaveWorkedWith', 'OfficeStackSyncHaveWorkedWith',
    'AISearchHaveWorkedWith', 'AIDevHaveWorkedWith', 'AIAcc',
    'AIBen', 'AIToolCurrently Using'
    ], axis=1)

In [244]:
for col in df.columns:
    print(col)

MainBranch
CodingActivities
EdLevel
LearnCode
YearsCode
YearsCodePro
OrgSize
PurchaseInfluence
CompTotal
AISelect
AISent
Frequency_1
Frequency_2
Frequency_3
TimeSearching
TimeAnswering
Age_18-24 years old
Age_25-34 years old
Age_35-44 years old
Age_45-54 years old
Age_55-64 years old
Age_65 years or older
Age_Under 18 years old
employment_map
RemoteWork_Hybrid (some remote, some in-person)
RemoteWork_In-person
RemoteWork_Remote
LearnCodeOnline_Category_Technical_Reference
LearnCodeOnline_Category_Visual_Interactive
DevType_Category_Desarrollo de Software
DevType_Category_Gestión y Liderazgo
DevType_Category_Hardware y Blockchain
DevType_Category_Infraestructura y Sistemas
DevType_Category_Ingeniería de Datos y Ciencia de Datos
DevType_Category_Investigación y Educación
DevType_Category_Roles de Soporte y Otros
Android-based
Linux-based
MacOS
Otros
Windows-based
Industry_Category_Industria y Energía
Industry_Category_Otros Servicios
Industry_Category_Salud y Educación
Industry_Category_

In [245]:
df.head()

,MainBranch,CodingActivities,EdLevel,LearnCode,YearsCode,YearsCodePro,OrgSize,PurchaseInfluence,CompTotal,AISelect,AISent,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Age_18-24 years old,Age_25-34 years old,Age_35-44 years old,Age_45-54 years old,Age_55-64 years old,Age_65 years or older,Age_Under 18 years old,employment_map,"RemoteWork_Hybrid (some remote, some in-person)",RemoteWork_In-person,RemoteWork_Remote,LearnCodeOnline_Category_Technical_Reference,LearnCodeOnline_Category_Visual_Interactive,DevType_Category_Desarrollo de Software,DevType_Category_Gestión y Liderazgo,DevType_Category_Hardware y Blockchain,DevType_Category_Infraestructura y Sistemas,DevType_Category_Ingeniería de Datos y Ciencia de Datos,DevType_Category_Investigación y Educación,DevType_Category_Roles de Soporte y Otros,Android-based,Linux-based,MacOS,Otros,Windows-based,Industry_Category_Industria y Energía,Industry_Category_Otros Servicios,Industry_Category_Salud y Educación,Industry_Category_Servicios Financieros,Industry_Category_Tecnología y Servicios Digitales
0,5,2.0,5,7,0,10,1,0,35000.0,2,4,1,1,1,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0.0,0.0,0.0,0.0,1.0
1,5,-1.0,3,6,15,15,8,0,70000.0,1,3,1,1,1,1,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0.0,0.0,0.0,0.0,1.0
2,5,-1.0,4,8,5,3,8,0,25000.0,1,0,0,1,0,2,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0.0,0.0,0.0,0.0,1.0
3,5,0.0,4,4,4,2,5,0,35000.0,2,3,-1,-1,-1,-1,-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0
4,5,-1.0,6,5,15,10,6,0,42000.0,2,4,-1,-1,-1,-1,-1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0


In [246]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1150 entries, 0 to 1149
Data columns (total 46 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   MainBranch                                               1150 non-null   int64  
 1   CodingActivities                                         1150 non-null   float64
 2   EdLevel                                                  1150 non-null   int64  
 3   LearnCode                                                1150 non-null   int64  
 4   YearsCode                                                1150 non-null   int32  
 5   YearsCodePro                                             1150 non-null   int32  
 6   OrgSize                                                  1150 non-null   int64  
 7   PurchaseInfluence                                        1150 non-null   int64  
 8   CompTotal                   

In [247]:
# Exporto este df final, para hacer concat con el de 2024 y trabajar con uno solo en el modelo. 
# Guardar el DataFrame en un archivo
with open('df2023.pickle', 'wb') as archivo:
    pickle.dump(df, archivo)